In [ ]:
pip install requests msal jsons azure-storage-blob requests_toolbelt

In [ ]:
from services.aadservice import AadService
aad_token = AadService.get_access_token()

In [ ]:
workspace_id="16453fea-cb0f-4ef4-b1b2-bede9d3b92be"
display_name="NYCYellow.pbix"

In [ ]:
from os import path
#file_location = 'files/AdventureWorks.pbix'
file_location = f'files/{display_name}'
file_size = path.getsize(file_location)


In [ ]:
import requests
from requests_toolbelt.multipart.encoder import MultipartEncoder

#This demonstrates loading files under 1GB
if file_size < 1073741824:
    print("File is less than 1GB this cell is loading the file as form data")

    url = f'https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}/imports?datasetDisplayName={display_name}'
    headers = {
        'Content-Type': 'multipart/form-data',
        'authorization': 'Bearer ' + aad_token
    }

    # you need this dictionary to convert a binary file into form-data format
    # None here means we skip the filename and file content is important 
    files = {'value': (None, open(file_location, 'rb'), 'multipart/form-data')}

    mp_encoder = MultipartEncoder(fields=files)
    print(mp_encoder)
    r = requests.post(
        url=url,
        data=mp_encoder,  # The MultipartEncoder is posted as data, don't use files=...!
        # The MultipartEncoder provides the content-type header with the boundary:
        headers=headers
    )
else:
    print("File is larger than 1GB this cell did nothing")


In [ ]:
import requests
import json
from azure.storage.blob import BlobClient

#This demonstrates loading files under 1GB
if file_size >= 1073741824:
    temp_url = ""

    #Part 1:  Request a temporary blob location from the API.
    api_url = f'https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}/imports/createTemporaryUploadLocation'
    headers =  {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer ' + aad_token
    } 

    api_response = requests.post(api_url, headers=headers)
    #error handling for createTemporaryUplodadLocation
    if api_response.status_code != 200:
        description = f'Error requesting temp location:\n  -Status Code:\t{api_response.status_code}\n  -Reason:\t{api_response.reason}\n  -RequestId:\t{api_response.headers.get("RequestId")}\n  -Text:\t{api_response.text}'
        print(description)
    else:
        api_response = json.loads(api_response.text)
        temp_url = api_response['url']

    #Part 2:  Now that we have the temporary location, upload the file to it via the BlobClient
    if temp_url != '':
        client = BlobClient.from_blob_url(temp_url)
        with open(file_location, 'rb') as data:
            client.upload_blob(data)
        print(f"{file_location} uploaded")
    else:
        print("No sas url specified.")

    #Part 3:   Call the import API with the temp location as 'fileUrl' in the body of the API call
    api_url = f"https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}/imports?datasetDisplayName={display_name}&nameConflict=Ignore&skipReport=false&overrideReportLabel=true&overrideModelLabel=true"

    body = {
        "fileUrl": temp_url
    }

    api_response = requests.post(
        url=api_url,
        headers=headers,
        data=json.dumps(body)
    )

    if api_response.status_code != 200:
        description = f'Error importing file:\n  -Status Code:\t{api_response.status_code}\n  -Reason:\t{api_response.reason}\n  -RequestId:\t{api_response.headers.get("RequestId")}\n  -Text:\t{api_response.text}'
        print(description)
    else:
        api_response = json.loads(api_response.text)
        print(json.dumps(api_response,indent=2))
